In [1]:
import pandas as pd
from pandasgui import show

In [2]:
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

# IMPORTING THE DATASETS

In [3]:
path_for_movies = 'movies.csv'
path_for_ratings = 'ratings.csv'

In [4]:
movies = pd.read_csv(path_for_movies)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings = pd.read_csv(path_for_ratings)
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# MERGING THE TWO DATASETS

In [6]:
ratings = pd.merge(movies,ratings)
ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


# DATA WRANGLING

In [7]:
ratings = ratings.drop(['genres','timestamp'],axis=1)
ratings

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5
...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),184,4.0
100832,193583,No Game No Life: Zero (2017),184,3.5
100833,193585,Flint (2017),184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5


In [8]:
user_ratings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
user_ratings.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#DROP ALL THE MOVIES THAT HAVE NOT BEEN RATED BY EVEN 10 USERS
user_ratings = user_ratings.dropna(thresh=10,axis=1).fillna(0)
guitext = user_ratings
user_ratings

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.5,3.5,0.0


In [10]:
import numpy as np
guitext = user_ratings.columns
guitext
guitext.to_numpy()
guitext = np.random.choice(guitext,200)
for i in guitext:
    print(i)

Before Sunset (2004)
Exorcist, The (1973)
Bull Durham (1988)
Hollow Man (2000)
Mission: Impossible (1996)
When Harry Met Sally... (1989)
Bridge on the River Kwai, The (1957)
Pitch Black (2000)
Uncle Buck (1989)
X-Men: Apocalypse (2016)
Harold & Kumar Escape from Guantanamo Bay (2008)
Creature Comforts (1989)
Life Aquatic with Steve Zissou, The (2004)
Bourne Legacy, The (2012)
Ghostbusters (a.k.a. Ghost Busters) (1984)
Good bye, Lenin! (2003)
Eternal Sunshine of the Spotless Mind (2004)
Horrible Bosses (2011)
Hedwig and the Angry Inch (2000)
Net, The (1995)
Tin Cup (1996)
Kill Bill: Vol. 1 (2003)
Ringu (Ring) (1998)
K-PAX (2001)
Terminator 3: Rise of the Machines (2003)
The Hateful Eight (2015)
Flipper (1996)
Forget Paris (1995)
Meet the Parents (2000)
Fahrenheit 451 (1966)
Monsters University (2013)
Swimming with Sharks (1995)
Big Short, The (2015)
Happy Gilmore (1996)
Intouchables (2011)
The Hunger Games: Catching Fire (2013)
Eight Men Out (1988)
Nine Months (1995)
Mission: Impossible

In [11]:
#NOW WE CHECK THE CORELATION BETWEEN DIFFERENT MOVIES USING PEARSON CORELATION
item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.063117,-0.023768,0.143482,0.011998,0.087931,0.224052,0.034223,0.009277,0.008331,...,0.017477,0.032470,0.134701,0.153158,0.101301,0.049897,0.003233,0.187953,0.062174,0.353194
(500) Days of Summer (2009),0.063117,1.000000,0.142471,0.273989,0.193960,0.148903,0.142141,0.159756,0.135486,0.200135,...,0.374515,0.178655,0.068407,0.414585,0.355723,0.252226,0.216007,0.053614,0.241092,0.125905
10 Cloverfield Lane (2016),-0.023768,0.142471,1.000000,-0.005799,0.112396,0.006139,-0.016835,0.031704,-0.024275,0.272943,...,0.242663,0.099059,-0.023477,0.272347,0.241751,0.195054,0.319371,0.177846,0.096638,0.002733
10 Things I Hate About You (1999),0.143482,0.273989,-0.005799,1.000000,0.244670,0.223481,0.211473,0.011784,0.091964,0.043383,...,0.243118,0.104858,0.132460,0.091853,0.158637,0.281934,0.050031,0.121029,0.130813,0.110612
"10,000 BC (2008)",0.011998,0.193960,0.112396,0.244670,1.000000,0.234459,0.119132,0.059187,-0.025882,0.089328,...,0.260261,0.087592,0.094913,0.184521,0.242299,0.240231,0.094773,0.088045,0.203002,0.083518


In [12]:
#RECOMMENDER
# import pandasgui
def get_similar_movies(movie_name,user_rating):
    similar_score = item_similarity_df[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False).head(10)
    
    return similar_score
    
print(get_similar_movies("2001: A Space Odyssey (1968)",5))


title
2001: A Space Odyssey (1968)                                                   2.500000
Blade Runner (1982)                                                            1.498907
Apocalypse Now (1979)                                                          1.206527
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)    1.176868
Clockwork Orange, A (1971)                                                     1.172016
Aliens (1986)                                                                  1.171963
Alien (1979)                                                                   1.170945
Brazil (1985)                                                                  1.155183
Fistful of Dollars, A (Per un pugno di dollari) (1964)                         1.116789
Close Encounters of the Third Kind (1977)                                      1.115427
Name: 2001: A Space Odyssey (1968), dtype: float64


In [ ]:
from kivy.clock import Clock

from kivymd.app import MDApp

from kivymd.uix.menu import MDDropdownMenu
from kivy.lang import Builder
from kivy.uix.screenmanager import Screen, ScreenManager


screen_helper = '''
#:import CardTransition kivy.uix.screenmanager.CardTransition
#:import FallOutTransition kivy.uix.screenmanager.FallOutTransition
#:import NoTransition kivy.uix.screenmanager.NoTransition

ScreenManager:
    LogoScreen:
    MovieScreen:
    RecommendationScreen:

<LogoScreen>
    name :'logoscreen'
    Image:
        source: 'sskenterprise.jpg'
        size: self.texture_size


<MovieScreen>:
    name:'Movie'
    FloatLayout:
        orientation: 'horizontal'
        canvas:
            Rectangle:
                source: 'bgblue.jpg'
                size: self.size
                pos: self.pos
    # MDCard:
    #     size_hint : (None, None)
    #     width:500
    #     height:400 
    #     # pos_hint: {"center_x": .5, "center_y": .5}
    #     pos:(root.width/6,root.height/6)
    #     padding:10
    

    MDLabel:
        text: "Welcome to the"
        halign:'center'
        font_size: "35px"
        color: rgba("#ffffff")
        # pos_hint: {'center_x': 0.6, 'center_y': 0.9}
        pos:(10,470)
        size_hint_y: None
        height: self.texture_size[1]

    MDLabel:
        text: "MOVIE MUNCHIES"
        halign:'center'
        font_size: "45px"
        color: rgba("#ffffff")
        # pos_hint: {'center_x': 0.6, 'center_y': 0.9}
        pos:(10,420)
        size_hint_y: None
        height: self.texture_size[1]

    MDLabel:
        text: "can't decide what to watch? well\\ndont worry as we have got you covered."
        halign:'center'
        font_size: "35px"
        color: rgba("#ffffff")
        # pos_hint: {'center_x': 0.6, 'center_y': 0.9}
        pos:(10,340)
        size_hint_y: None
        height: self.texture_size[1]


    MDTextField:
        id: field
        
        dropdown_bg: app.theme_cls.bg_normal
        
        # pos_hint: {'center_x': 0.5, 'center_y': 0.5}
        size_hint_x: None
        width: "200dp"
        hint_text: "Movie Names"
        pos:(100,250)
        on_focus: if self.focus: app.menu.open()
        
    MDTextField:
        id: ratings
        
        dropdown_bg: app.theme_cls.bg_normal
        size_hint_x: None
        width: "200dp"
        hint_text: "Rating"
        pos:(500,250)
        on_focus: if self.focus: app.rating_menu.open()
        color: 1,0,1,1
    
    Button:
        text:'FIND'
        font_color: rgba("#000000")
        font_size:24
        # background_normal: ''
        background_color: rgba("#ffffff")
        size: 250, 50
        size_hint: None, None
        pos:(250,60)
        on_press:
            #root.manager.current = 'Recommendation'
            app.get_my_movies()

            
<RecommendationScreen>:
    name:'Recommendation'
    MDLabel:
        text:'Welcome to profile'
        halign:'center'


'''

class LogoScreen(Screen):
    pass


class MovieScreen(Screen):
    pass


class RecommendationScreen(Screen):
    pass


class timer():
    def work1(self):
        print("Hello World")


sm = ScreenManager()
sm.add_widget(LogoScreen(name='logoscreen'))
sm.add_widget(MovieScreen(name='Movie'))
sm.add_widget(RecommendationScreen(name='Recommendation'))

def callbackfun(obj):
    MDApp.get_running_app().root.current = 'Movie'


class MovieRecommender(MDApp):
    def __init__(self):
        super().__init__()
        self.screen = Builder.load_string(screen_helper)
        self.theme_cls.primary_palette = 'Gray'
        self.theme_cls.theme_style = "Light"
        Clock.schedule_once(self.for_movies)
        Clock.schedule_once(self.for_ratings)
        self.movie_name = ''
        self.user_ratings = ''

    def build(self):
        # loading_screen()
        Clock.schedule_once(callbackfun, 4)

        return self.screen

    def for_movies(self, *args):
        menu_items = [{"text": f"{i}"} for i in guitext]
        self.menu = MDDropdownMenu(
            caller=self.screen.get_screen('Movie').ids.field,
            items=menu_items,
            position="bottom",
            width_mult=4,
            callback=self.set_item
        )
        self.menu.bind(on_release=self.set_item)

    def set_item(self, instance):
        self.screen.get_screen('Movie').ids.field.text = instance.text
        self.movie_name = instance.text


    def for_ratings(self, *args):
        ratings_menu_items = [{"text": f"{i}"} for i in range(1,6)]
        self.rating_menu = MDDropdownMenu(
            caller=self.screen.get_screen('Movie').ids.ratings,
            items=ratings_menu_items,
            position="bottom",
            width_mult=4,
            callback=self.set_ratings
        )
        self.menu.bind(on_release=self.set_ratings)

    def set_ratings(self, instance):
        self.screen.get_screen('Movie').ids.ratings.text = instance.text
        self.user_ratings = float(instance.text)
        
    def get_similar_movies(self,mn,ur):
        similar_score = item_similarity_df[mn]*(ur-2.5)
        similar_score = similar_score.sort_values(ascending=False).head(10)

        return similar_score

    def get_my_movies(self):
        show(get_similar_movies(self.movie_name,self.user_ratings))
        

    

MovieRecommender().run()


[INFO   ] [Logger      ] Record log in C:\Users\shehryar mallick\.kivy\logs\kivy_21-01-23_123.txt
PandasGUI INFO — kivy — [Logger      ] Record log in C:\Users\shehryar mallick\.kivy\logs\kivy_21-01-23_123.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.3.0
PandasGUI INFO — kivy — [deps        ] Successfully imported "kivy_deps.angle" 0.3.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.3.0
PandasGUI INFO — kivy — [deps        ] Successfully imported "kivy_deps.glew" 0.3.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.3.1
PandasGUI INFO — kivy — [deps        ] Successfully imported "kivy_deps.sdl2" 0.3.1
[INFO   ] [Kivy        ] v2.0.0
PandasGUI INFO — kivy — [Kivy        ] v2.0.0
[INFO   ] [Kivy        ] Installed at "C:\Users\shehryar mallick\anaconda3\envs\MLCourse\lib\site-packages\kivy\__init__.py"
PandasGUI INFO — kivy — [Kivy        ] Installed at "C:\Users\shehryar mallick\anaconda3\envs\MLCourse\lib\site-packages\ki